In [ ]:
import psycopg2
import sys
from sqlalchemy import create_engine
import geopandas as gpd

In [ ]:
DATABASE_NAME = "lab_postgis"

In [ ]:
def getEngineConnectionString(user='postgres', password='postgre987', dbname='postgres'):
    return "postgresql://" + user + ":" + password + "@localhost/" + dbname

In [ ]:
def getOpenConnection(user='postgres', password='postgre987', dbname='postgres'):
    return psycopg2.connect(database = dbname, user = user, host='localhost', password= password)

In [ ]:
def createDB(dbname='postgres'):
    """
    We create a DB by connecting to the default user and database of Postgres
    The function first checks if an existing database exists for a given name, else creates it.
    :return:None
    """
    # Connect to the default database
    con = getOpenConnection(dbname='postgres')
    con.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)
    cur = con.cursor()

    # Check if an existing database with the same name exists
    cur.execute('SELECT COUNT(*) FROM pg_catalog.pg_database WHERE datname=\'%s\'' % (dbname,))
    count = cur.fetchone()[0]
    if count == 0:
        cur.execute('CREATE DATABASE %s' % (dbname,))  # Create the database
        print("DATABASE " + dbname + " created")
    else:
        print("Database with the given name already exists")

    # Clean up
    cur.close()
    con.commit()
    con.close()

In [ ]:
createDB(DATABASE_NAME)

In [ ]:
engine = create_engine(getEngineConnectionString(dbname = DATABASE_NAME))
con = engine.connect()
print("sqlalchemy engine created")

In [ ]:
try:
    con.execute("CREATE EXTENSION postgis;") # Add PostGIS extension
    print("postgis extension created")
except:
    print("Unable to create postgis extension")

In [ ]:
dfAirbnb = gpd.read_file("inputs/airbnb")

In [ ]:
dfAirbnb.head()

In [ ]:
dfAirbnb.to_postgis("airbnb", engine)#(tablename, connection_object)

In [ ]:
#-------------------------------------------------------------------------------------------------------------------------#

In [ ]:
dfGrocery = gpd.read_file("inputs/grocery/Chicago_sup.shp")

In [ ]:
dfGrocery.head()

In [ ]:
dfGrocery.to_postgis("grocery", engine)

In [ ]:
#close the previous connection and open a new connection where we specify the name of our database
#so this database will be used for further execution
if con:
    con.close()

con = getOpenConnection(dbname = DATABASE_NAME)
cur = con.cursor()

In [ ]:
#We need to check if SRIDs of geometry column of both tables are same
#get the SRID for Airbnb table
cur.execute("Select ST_SRID(geometry) from airbnb limit 1")
results = cur.fetchall()
print(results)

In [ ]:
#get the SRID of grocery table
cur.execute("Select ST_SRID(geometry) from grocery limit 1")
results = cur.fetchall()
print(results)

In [ ]:
#change SRID for grocery so that both are compatible for further operations
cur.execute("create table grocery3 as select ST_Transform(geometry, 4326) as geometry from grocery")

In [ ]:
cur.execute("select ST_AsText(geometry) from grocery3 limit1")
results = cur.fetchall()
print(results)

In [ ]:
cur.execute("Select ST_SRID(geometry) from grocery3 limit 1")
results = cur.fetchall()
print(results)

In [ ]:
#actual problem with solution
#count how many grocery shops are there in the neighbourhood of an airbnb
cur.execute("CREATE TABLE airbnb_grocery3 AS SELECT airbnb.geometry AS geom1, COUNT(grocery2.geometry) AS grocery_count FROM airbnb, grocery2 WHERE ST_Distance(airbnb.geometry, grocery2.geometry) < 133.9 GROUP BY airbnb.geometry")
#above query has processed the calculation and stored the result in a new table airbnb_grocery

In [ ]:
#print the results of calculation in above cell
cur.execute("SELECT * FROM airbnb_grocery3 limit 5")
result = cur.fetchall()
print(results)

In [ ]:
cur.close()
con.commit()
con.close()

In [ ]:
help(engine)